In [1]:
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
conf.set("spark.app.name", "natasha pritykovskaya Hyperparameters") 

spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [3]:
spark

In [7]:
import pandas as pd

In [8]:
pdf = pd.read_csv("diamonds.csv", header=0).iloc[:,1:]

In [9]:
pdf.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


## Feature engineering

In [10]:
pdf['cut'] = pdf['cut'].replace({'Fair': 0, 'Good': 1, 'Very Good': 2, 'Premium': 3, 'Ideal': 4})
pdf['color'] = pdf['color'].replace({'J': 0, 'I': 1, 'H': 2, 'G': 3, 'F': 4, 'E': 5, 'D': 6})
pdf['clarity'] = pdf['clarity'].replace({'I1': 0, 'SI1': 1, 'SI2': 2, 'VS1': 3, 'VS2': 4, 'VVS1': 5, 'VVS2': 6, 'IF': 7})
pdf.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,4,5,2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,3,5,1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,1,5,3,56.9,65.0,327,4.05,4.07,2.31
3,0.29,3,1,4,62.4,58.0,334,4.20,4.23,2.63
4,0.31,1,0,2,63.3,58.0,335,4.34,4.35,2.75


In [11]:
labels = pdf['price'].values
featureNames = ['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z']
features = pdf[featureNames].values

In [12]:
from sklearn.preprocessing import normalize
features = normalize(features, axis=0)
features

array([[0.00106702, 0.00553547, 0.005655  , ..., 0.0029123 , 0.00293078,
        0.00289958],
       [0.00097424, 0.0041516 , 0.005655  , ..., 0.00286806, 0.00282769,
        0.00275639],
       [0.00106702, 0.00138387, 0.005655  , ..., 0.00298603, 0.00299705,
        0.00275639],
       ...,
       [0.00324745, 0.00276773, 0.006786  , ..., 0.00417307, 0.00418262,
        0.00424794],
       [0.00398973, 0.0041516 , 0.002262  , ..., 0.00453434, 0.00450662,
        0.00446272],
       [0.00347941, 0.00553547, 0.006786  , ..., 0.0042984 , 0.00432253,
        0.0043434 ]])

## Parameter tuning со Spark

Parameter tuning - это задача тьюнинга (гипер) параметров ML алгоритма с целью повысить качество модели. Тренируются различные модели (каждая со своим набором параметров) на одном и том же наборе данных, и далее все полученные модели тестируются на одном и том же отложенном наборе данных, что снижает риск переобучения.

k-fold cross validation:


 - Случайным образом разбиваем данные на к равных частей ("folds")
     -  i = 1, 2, ..., k, откладываем набор данных i как validation set.
     -  training set - все кроме i

     -  для каждого набора параметров тренируем модель, подсчитываем ошибку на k различных validation set, усредняем, находим лучший набор параметров

 - Тренируем модель с лучшим набором параметров на всех данных 


Для каждой пары (fold, parameter set) можно обучать модель независимо от всех остальных. Мы распараллелим эти задания: scikit-learn будет обучать модель на каждом executor'е. Это параллелизация очень эффективна, так как обучение моделей - самая вычислительно сложная часть ML workflow.

Если используются k фолдов и P различных наборов параметров, то во сколько раз можно ускорить вычисление при неограниченном кол-ве ресурсов?


### Отложим random test set


In [13]:
import sklearn

In [14]:
from sklearn.model_selection import train_test_split

trainingLabels, testLabels, trainingFeatures, testFeatures = train_test_split(labels, features, test_size=0.3)
ntrain, ntest = len(trainingLabels), len(testLabels)
print('Split data randomly into 2 sets: %d training and %d test instances.' % (ntrain, ntest))

Split data randomly into 2 sets: 37758 training and 16182 test instances.


### Разобьем данные и определим таски, которые будем параллелизировать
Каждое распределенное задание это пара - (fold, parameter set) pair.

In [15]:
from sklearn.model_selection import KFold
numFolds = 3 # more (10 or so) in practice
kf = KFold(n_splits=numFolds)

In [34]:
for trainIndex_, valIndex_  in kf.split(trainingFeatures):
    print("Shape train " + str(trainingFeatures[trainIndex_].shape))
    print("Shape val " +  str(trainingFeatures[valIndex_].shape))

Shape train (25172, 9)
Shape val (12586, 9)
Shape train (25172, 9)
Shape val (12586, 9)
Shape train (25172, 9)
Shape val (12586, 9)


In [17]:
alphas = [0.0, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
tasks = []
for alpha in alphas:
    for fold in range(numFolds):
        tasks = tasks + [(alpha, fold)]

In [18]:
tasks

[(0.0, 0),
 (0.0, 1),
 (0.0, 2),
 (0.0001, 0),
 (0.0001, 1),
 (0.0001, 2),
 (0.001, 0),
 (0.001, 1),
 (0.001, 2),
 (0.01, 0),
 (0.01, 1),
 (0.01, 2),
 (0.1, 0),
 (0.1, 1),
 (0.1, 2),
 (1.0, 0),
 (1.0, 1),
 (1.0, 2),
 (10.0, 0),
 (10.0, 1),
 (10.0, 2),
 (100.0, 0),
 (100.0, 1),
 (100.0, 2),
 (1000.0, 0),
 (1000.0, 1),
 (1000.0, 2)]

In [19]:
len(tasks)

27

In [21]:
sc = spark.sparkContext

In [22]:
tasksRDD = sc.parallelize(tasks, numSlices = len(tasks))

In [23]:
tasksRDD.getNumPartitions()

27

### Broadcast dataset

In [24]:
trainingFeaturesBroadcast = sc.broadcast(trainingFeatures)
trainingLabelsBroadcast = sc.broadcast(trainingLabels)

### Запустим параллельную cross-validation

Определим функцию, которая будет запускаться на каждом worker'e, эта функция берет одну пару (1 hyperparameter alpha value + 1 fold index) и тренируем соотвествующую модель. Используем RDD.map для этого.

In [35]:
from sklearn import linear_model

def trainOneModel(alpha, fold):
    """
    Given 1 task (1 hyperparameter alpha value + 1 fold index), 
    train the corresponding model.
    
    Return: model, score on the fold's test data, task info.
    """
    localTrainingFeatures = trainingFeaturesBroadcast.value
    localTrainingLabels = trainingLabelsBroadcast.value
    trainIndex, valIndex = [], []
    fold_ = 0 
    
    for trainIndex_, valIndex_ in kf.split(localTrainingFeatures):
        if fold_ == fold:
            trainIndex, valIndex = trainIndex_, valIndex_
            break
        fold_ += 1
    X_train, X_val = localTrainingFeatures[trainIndex], localTrainingFeatures[valIndex]
    Y_train, Y_val = localTrainingLabels[trainIndex], localTrainingLabels[valIndex]

    clf = linear_model.Ridge(alpha=alpha)
    clf.fit(X_train, Y_train)
    score = clf.score(X_val, Y_val)
    return clf, score, alpha, fold

In [36]:
trainedModelAndScores = tasksRDD.map(lambda alpha_fold: trainOneModel(alpha_fold[0], alpha_fold[1]))
trainedModelAndScores.cache()
trainedModelAndScores.count()

27

In [37]:
trainingFeaturesBroadcast.unpersist()
trainingLabelsBroadcast.unpersist()

### Соберем результаты для лучшей hyperparameter alpha

In [38]:
allScores = trainedModelAndScores.map(lambda x: (x[1], x[2], x[3])).collect()
avgScores = dict(map(lambda alpha: (alpha, 0.0), alphas))

In [39]:
allScores

[(0.8949694972327946, 0.0, 0),
 (0.8899219450629374, 0.0, 1),
 (0.892972403259172, 0.0, 2),
 (0.8950170602987831, 0.0001, 0),
 (0.8925033567715434, 0.0001, 1),
 (0.892670797821387, 0.0001, 2),
 (0.8929740935442487, 0.001, 0),
 (0.8894317800192453, 0.001, 1),
 (0.8906300460432877, 0.001, 2),
 (0.8797613010847548, 0.01, 0),
 (0.8742826032125341, 0.01, 1),
 (0.8794463751181155, 0.01, 2),
 (0.7320493489333039, 0.1, 0),
 (0.719806710665414, 0.1, 1),
 (0.7343544125160528, 0.1, 2),
 (0.22759389448245715, 1.0, 0),
 (0.22325716556097774, 1.0, 1),
 (0.22981292673388476, 1.0, 2),
 (0.02833874911638734, 10.0, 0),
 (0.02751163418803171, 10.0, 1),
 (0.028507979737752317, 10.0, 2),
 (0.002879267184887624, 100.0, 0),
 (0.0024678450328748935, 100.0, 1),
 (0.0027261788463880388, 100.0, 2),
 (0.0002635102993848726, 1000.0, 0),
 (-0.00010567104145153827, 1000.0, 1),
 (7.658548496891626e-05, 1000.0, 2)]

In [40]:
avgScores

{0.0: 0.0,
 0.0001: 0.0,
 0.001: 0.0,
 0.01: 0.0,
 0.1: 0.0,
 1.0: 0.0,
 10.0: 0.0,
 100.0: 0.0,
 1000.0: 0.0}

In [41]:
for score, alpha, fold in allScores:
    avgScores[alpha] += score
for alpha in alphas:
    avgScores[alpha] /= numFolds
avgScores

{0.0: 0.8926212818516347,
 0.0001: 0.8933970716305711,
 0.001: 0.8910119732022604,
 0.01: 0.877830093138468,
 0.1: 0.7287368240382568,
 1.0: 0.22688799559243988,
 10.0: 0.028119454347390455,
 100.0: 0.0026910970213835186,
 1000.0: 7.814158096741686e-05}

Теперь у нас есть список alpha values с соотвествующими средними scores, найдем среди них лучший.

In [42]:
# Find best score
bestAlpha = -1
bestScore = -1
for alpha in alphas:
    if avgScores[alpha] > bestScore:
        bestAlpha = alpha
        bestScore = avgScores[alpha]
print('Found best alpha: %g, which gives score: %g' % (bestAlpha, bestScore))

Found best alpha: 0.0001, which gives score: 0.893397


### Обучим финальную модель с лучшим набором гиперпараметров

Так это только 1 таск, то запустим его на драйвере.

In [43]:
# Use bestAlpha, and train a final model.
tunedClf = linear_model.Ridge(alpha=bestAlpha)
tunedClf.fit(trainingFeatures, trainingLabels)

Ridge(alpha=0.0001, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [ ]:
spark.stop()